### Libs & Data

In [1]:
import pandas as pd
import numpy as np
import os
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import string
import random
import unicodedata
from string import punctuation
from string import digits
from nltk.stem import WordNetLemmatizer
import joblib

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import PCA
# from sklearn.decomposition import TruncatedSVD
# from sklearn.manifold import TSNE
# from sklearn.cluster import KMeans
# from sklearn.cluster import DBSCAN
# from sklearn.neighbors import NearestNeighbors
# from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt

import torch
# from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

import spacy
from spacy.matcher import Matcher

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kwsst\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kwsst\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df2015 = pd.read_csv("data/df2015.csv")
df2016 = pd.read_csv("data/df2016.csv")
df2017 = pd.read_csv("data/df2017.csv")
df2017.head()

In [2]:
df = pd.read_csv("data/top20sum_sents.csv")
df.head()

,2015,label2015,freq2015,2016,label2016,freq2016,2017,label2017,freq2017,sents2015,sents2016,sents2017
0,South Korea reported no additional cases of Mi...,2957,6,"Radio Pyongyang, the North's state-run radio s...",16,5,Temperatures across the country plummeted abou...,67,11,South Korea reported no additional cases of Mi...,"Radio Pyongyang, the North's state-run radio s...",Temperatures across the country plummeted abou...
1,"In October, some 36,900 babies were born, up 1...",132,5,Some 200 chickens were found dead on Monday mo...,518,4,A South Korean research team says it has uncov...,617,7,"In October, some 36,900 babies were born, up 1...",Some 200 chickens were found dead on Monday mo...,A South Korean research team says it has uncov...
2,"Cho Hyun-ah, former vice president of Korean A...",6018,4,Voter turnout in South Korea’s parliamentary e...,5095,4,South Korean scientists have developed an adhe...,4735,5,"Cho Hyun-ah, former vice president of Korean A...",Just less than 60 percent of South Korean vote...,A group of South Korean scientists have develo...
3,Seoul-Tokyo ties have plunged to lowest levels...,5457,4,Walkway along Deoksu Palace in downtown Seoul ...,705,3,"About 30,300 babies were born in May, down 11....",3708,5,South Korea and Japan held talks on Tokyo's se...,Some 100 meters of walkway by the famous stone...,"About 30,300 babies were born in May, down 11...."
4,South Korea's top financial regulator said Fri...,5039,4,Pigs at two swine farms in Nonsan in the centr...,5782,3,"The search for the missing South Korean ship, ...",6654,4,South Korea's top financial regulator said Fri...,Pigs at two swine farms in Nonsan in the centr...,Brazilian Air Force has mobilized an aircraft ...


### LDA on summarized body

In [ ]:
def spacy_tokenizer(s: str):
    # Change similar terms to the same term
    new_str = text_cleaning(s)
    doc = nlp(s)
    # Group tokens
    matcher = Matcher(nlp.vocab)
    token_groupup_pattern = [
        [{"LOWER": "the"}, {"LOWER": "united"}, {"LOWER": "nations"}],
        [{"LOWER": "the"}, {"LOWER": "united"}, {"LOWER": "states"}],
        [{"LOWER": "north"}, {"LOWER": "korea"}],
        [{"LOWER": "south"}, {"LOWER": "korea"}],
    ]
    matcher.add("TermGroup",token_groupup_pattern)
    matches = matcher(doc)
    merge_doc = []
    for nid, start, end in matches:
        merge_doc.append((start,end))
    with doc.retokenize() as retokenizer:
        for i in range(len(merge_doc)-1,-1,-1):
            retokenizer.merge(doc[merge_doc[i][0]:merge_doc[i][1]])
        
    # Remove all stopword, punctuation, number
    tokens = [ token.lemma_.lower() for token in doc \
              if not token.is_stop and not token.is_punct and not token.like_num and token.lemma_.strip()!= '']
    return tokens

In [ ]:
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, min_df=2)
all_texts = df['sents2015']
data_vectorized = vectorizer.fit_transform(tqdm(all_texts))





In [ ]:
def print_top_words(model, vectorizer, n_top_words):
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        message = "\nTopic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [ ]:
# Load model LDA
lda2015 = joblib.load('data/lda_title_2015.csv')
lda2016 = joblib.load('data/lda_title_2016.csv')
lda2017 = joblib.load('data/lda_title_2017.csv') 

In [ ]:
print_top_words(lda2015, vectorizer, n_top_words=25)

In [ ]:
print_top_words(lda2016, vectorizer, n_top_words=25)

In [ ]:
print_top_words(lda2017, vectorizer, n_top_words=25)